In [ ]:
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
import base64
import os

In [ ]:
token = ""

In [ ]:
headers = {
    "Authorization" : "token {}".format(token),
    "Accept" : "application/vnd.github.sailor-v-preview+json"
}

In [ ]:
def mod_file(new_file_name, mesg):
     with open(new_file_name, "w") as f:
         f.write(mesg + "\n")

In [ ]:
def get_encode(mesg):
    st_bytes = mesg.encode("ascii")
    bs64 = base64.b64encode(st_bytes)
    res = bs64.decode("ascii")
    return res

In [ ]:
def do_fork(owner, repo):
    url = "https://api.github.com/repos/" + owner + "/" + repo + "/forks"
    r = requests.post(url,headers=headers)

In [ ]:
def upload_file(self_username, repo, new_file_name, msg):
    upload_data = {
        "message" : "Update File!!",
        "content" : msg
    }
    url = "https://api.github.com/repos/" + self_username +"/" + repo + "/contents/" + new_file_name
    requests.put(url,data=json.dumps(upload_data),headers=headers)

In [ ]:
def make_pr_and_print(self_username, owner, repo, dep_name, v1, v2, new_file_name):
    st1 = "UPDATE " + str(dep_name) + " from " + str(v1) + " to " + str(v2) + "."
    msg = get_encode(st1)
    upload_file(self_username, repo, new_file_name, msg)
    mod_file(new_file_name, st1)
    st2 = self_username + ":main"
    data= {
        "title" : "UPDATE THE GIVEN DEPENDENCY",
        "body" : st1 ,
        "head" : st2,
        "base" : "main"
    }
    url = "https://api.github.com/repos/{}/{}/pulls".format(owner,repo)
    requests.post(url,data=json.dumps(data), headers=headers)
    r = requests.get(url,data=json.dumps(data), headers=headers)
    for i in r.json():
        usr_nm = i['head']['label']
        usr_ls = usr_nm.split(":")
        if(usr_ls[0] == self_username):
            print(i['_links']['html']['href'])
            return i['_links']['html']['href']

In [ ]:
def ver(st):
  v_lst = []
  temp = 0
  for i in range(len(st)):
    if(st[i] == "."):
      v_lst.append(temp)
      temp = 0
    else:
      temp = temp*10 + int(st[i])
  return v_lst

In [ ]:
def cmp(ver1, ver2):
  if(len(ver1) > len(ver2)):
    for i in range(len(ver1) - len(ver2)):
      ver2.append(0)
  elif(len(ver2) > len(ver1)):
    for i in range(len(ver2) - len(ver1)):
      ver1.append(0)
  for i in range(len(ver1)):
    if(ver1[i] > ver2[i]):
      return 1
    elif(ver2[i] > ver1[i]):
      return 0
  return 1;

#Enter the COMMAND

In [ ]:
lst = input("Enter the command").split(" ")

Enter the commandmyawesometool -update -i input.csv axios@0.23.9


In [ ]:
update_flag = 0
if(lst[1][1] == "u"):
    update_flag = 1
dep = ""
file_name = ""
if(update_flag == 1):
    file_name = lst[3]
    dep = lst[4]
else:
    file_name = lst[2]
    dep = lst[3]
dep_lst = dep.split("@")
dep_name = dep_lst[0]
dep_version = dep_lst[1]
version_status = []

In [ ]:
best_version = ver(dep_version)

In [ ]:
owner = []
repo = []
self_username = ""
new_file_name = "mo.txt"

In [ ]:
df = pd.read_csv(file_name)
df

,name,repo
0,dyte-react-sample-app,https://github.com/dyte-in/react-sample-app/
1,dyte-js-sample-app,https://github.com/dyte-in/javascript-sample-app
2,dyte-sample-app-backend,https://github.com/dyte-in/backend-sample-app


In [ ]:
def github_read_file(username, repository_name, file_path, github_token=None):
    headers = {}
    if github_token:
        headers['Authorization'] = f"token {github_token}"      
    url = f'https://api.github.com/repos/{username}/{repository_name}/contents/{file_path}'
    r = requests.get(url, headers=headers)
    r.raise_for_status()
    data = r.json()
    file_content = data['content']
    file_content_encoding = data.get('encoding')
    if file_content_encoding == 'base64':
        file_content = base64.b64decode(file_content).decode()
    return file_content
def scrape(github_token, username, repository_name, file_path):
    file_content = github_read_file(username, repository_name, file_path, github_token=github_token)
    data = json.loads(file_content)
    return data['dependencies'][dep_name][1:]

In [ ]:
repo_version = []
link = []
cnt = 0
file_path = 'package.json'
for i in df['repo']:
    link.append(i)
    cnt += 1
    tmp = i.split("/")
    owner.append(tmp[3])
    repo.append(tmp[4])

In [ ]:
for i in range(cnt):
    repo_version.append(scrape(token, owner[i], repo[i], file_path))

In [ ]:
repo_version

['0.24.0', '0.21.1', '0.23.0']

In [ ]:
version_status = []
for i in range(len(repo_version)):
  v1 = ver(repo_version[i])
  if(cmp(v1, best_version) == 0):
    version_status.append("false")
  else:
    version_status.append("true")

In [ ]:
version_status

['true', 'false', 'true']

In [ ]:
df['version'] = repo_version
df['version_satisfied'] = version_status

In [ ]:
df_first = df
df_first

,name,repo,version,version_satisfied
0,dyte-react-sample-app,https://github.com/dyte-in/react-sample-app/,0.24.0,true
1,dyte-js-sample-app,https://github.com/dyte-in/javascript-sample-app,0.21.1,false
2,dyte-sample-app-backend,https://github.com/dyte-in/backend-sample-app,0.23.0,true


In [ ]:
df_first.to_csv('first.csv')

In [ ]:
if(update_flag == 1):
    pr_link = []
    for i in range(cnt):
        if(version_status[i] == "false"):
            do_fork(owner[i], repo[i])
#             upload_file(self_username, repo[i], new_file_name)
            pr_link.append(make_pr_and_print(self_username, owner[i], repo[i], dep_name, repo_version[i], dep_version, new_file_name))
        else:
            pr_link.append("")
    df['update_pr'] = pr_link
    df_second = df
    df_second
    df_second.to_csv('upgrade.csv')

https://github.com/dyte-in/javascript-sample-app/pull/341


In [ ]:
#df_second

,name,repo,version,version_satisfied,update_pr
0,dyte-react-sample-app,https://github.com/dyte-in/react-sample-app/,0.24.0,true,
1,dyte-js-sample-app,https://github.com/dyte-in/javascript-sample-app,0.21.1,false,https://github.com/dyte-in/javascript-sample-a...
2,dyte-sample-app-backend,https://github.com/dyte-in/backend-sample-app,0.23.0,true,
